In [16]:
import torch
import re
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from PIL import Image
import requests
from io import BytesIO
import json

In [18]:
RAG_KNOWLEDGE_BASE = {
    "kinoa salatası": "Kinoa salatası yaklaşık 250 kaloridir. İçinde bol lif, demir ve magnezyum bulunur. Glütensizdir.",
    "avokado tostu": "Bir dilim avokado tostu ortalama 300-350 kaloridir. Sağlıklı yağlar açısından zengindir.",
    "tavuk suyu çorba": "Bir kase tavuk suyu çorba yaklaşık 80-100 kaloridir. Grip ve soğuk algınlığına iyi gelir.",
    "mercimek çorbası": "Bir kase mercimek çorbası ortalama 160 kaloridir. Bitkisel protein ve lif kaynağıdır.",
    "ızgara köfte": "Bir porsiyon (4 adet) ızgara köfte yaklaşık 200-250 kaloridir. Yüksek protein içerir.",
    "haşlanmış yumurta": "Bir adet büyük boy haşlanmış yumurta 75 kaloridir. En kaliteli protein kaynaklarından biridir.",
    "lahmacun": "Bir adet lahmacun ortalama 450 kaloridir. İnce hamurlu olduğu için diğer hamur işlerine göre daha hafiftir.",
    "muz": "Orta boy bir muz yaklaşık 105 kaloridir. Potasyum açısından zengindir ve enerji verir.",
    "pizza": "Bir dilim karışık pizza ortalama 285 kaloridir. Karbonhidrat ve yağ oranı yüksektir.",
    "türk kahvesi": "Şekersiz bir fincan Türk kahvesi yaklaşık 5-10 kaloridir. Metabolizmayı hızlandırmaya yardımcı olur."
}

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Çalışma Ortamı: {device.upper()}")

Çalışma Ortamı: CUDA


In [15]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

Unrecognized keys in `rope_parameters` for 'rope_type'='default': {'mrope_section'}


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Current model requires 336 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.


Loading weights:   0%|          | 0/729 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [ ]:
def retrieve_knowledge(query):
    """
    Basit bir Retrieval (Geri Getirme) fonksiyonu.
    Agent 'SEARCH' kararı verirse burası çalışır.
    """
    print(f"\n[SİSTEM] Veritabanında aranıyor: '{query}'...")

    results = []
    for key, value in RAG_KNOWLEDGE_BASE.items():
        if key in query.lower():
            results.append(value)
    
    if results:
        return "\n".join(results)
    else:
        return "Veritabanında bu besinle ilgili spesifik kalori bilgisi bulunamadı."

In [ ]:
def agentic_rag_pipeline(user_query, image=None):
    """
    Bu fonksiyon 'Reasoning Loop' (Mantık Döngüsü) kurar.
    Model önce düşünür, gerekirse arama yapar, sonra cevap verir.
    """
    
    # DECISION STEP
    system_instruction = """Sen otonom bir beslenme asistanısın.
    Kullanıcının sorusuna cevap vermek için dışarıdan net bir bilgiye (kalori, besin değeri vb.) ihtiyacın varsa:
    Çıktı olarak sadece 'ACTION: SEARCH [aranacak_terim]' yaz.
    
    Eğer genel bir sohbetse veya bilgiye zaten sahipsen, direkt cevabı yaz.
    """
    
    conversation = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": user_query}
    ]
    
    if image:
        conversation[1]["content"] = [
            {"type": "image", "image": image},
            {"type": "text", "text": user_query},
        ]

    text_prompt = processor.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    
    # Inference
    inputs = processor(text=[text_prompt], images=[image] if image else None, return_tensors="pt", padding=True).to(device)
    output = model.generate(**inputs, max_new_tokens=100)
    response = processor.batch_decode(output, skip_special_tokens=True)[0].split("assistant")[-1].strip()
    
    print(f"[AGENT DÜŞÜNCESİ]: {response}")

    # TOOL USE
    if "ACTION: SEARCH" in response:
        search_term = response.replace("ACTION: SEARCH", "").strip()
        
        rag_data = retrieve_knowledge(search_term)
        
        # SYNTHESIS
        final_prompt_content = f"""
        Kullanıcı Sorusu: {user_query}
        
        Veritabanından Bulunan Bilgi (RAG Context):
        {rag_data}
        
        Bu bilgiyi kullanarak kullanıcıya nazik ve açıklayıcı bir cevap ver.
        """
        
        final_conversation = [
             {"role": "user", "content": final_prompt_content}
        ]
        
        final_text = processor.apply_chat_template(final_conversation, tokenize=False, add_generation_prompt=True)
        inputs = processor(text=[final_text], images=None, return_tensors="pt", padding=True).to(device)
        
        final_output = model.generate(**inputs, max_new_tokens=200)
        final_answer = processor.batch_decode(final_output, skip_special_tokens=True)[0].split("assistant")[-1].strip()
        
        return final_answer
    
    else:
        # Model aramaya gerek duymadı, doğrudan cevabı döndür
        return response

In [19]:
print("-" * 50)
print("SENARYO 1: Veritabanı Sorgusu Gereken Durum")
soru1 = "Diyetim için avokado tostu yemek istiyorum, kaç kalori?"
cevap1 = agentic_rag_pipeline(soru1)
print(f"\n[NİHAİ CEVAP]: {cevap1}")

--------------------------------------------------
SENARYO 1: Veritabanı Sorgusu Gereken Durum
[AGENT DÜŞÜNCESİ]: ACTION: SEARCH [avokado tostu kalori]

[SİSTEM] Veritabanında aranıyor: '[avokado tostu kalori]'...

[NİHAİ CEVAP]: Avokado tostu ortalama 300-350 kalori içerir. Sağlıklı yağlar açısından zengindir.


In [ ]:
print("-" * 50)
print("SENARYO 1: Veritabanı Sorgusu Gereken Durum")
soru1 = "Diyetim için avokado tostu ve 2 ızgara tavuk yemek istiyorum, kaç kalori?"
cevap1 = agentic_rag_pipeline(soru1)
print(f"\n[NİHAİ CEVAP]: {cevap1}")

--------------------------------------------------
SENARYO 1: Veritabanı Sorgusu Gereken Durum
[AGENT DÜŞÜNCESİ]: ACTION: SEARCH [avokado tostu kalori]

[SİSTEM] Veritabanında aranıyor: '[avokado tostu kalori]'...
